# Use this code to predict kindey boundaries based on the trained models from section (2)

In [1]:
from keras.models import Sequential, load_model
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
from keras_unet.metrics import dice_coef
from keras_unet.models import custom_unet
from keras_unet.losses import jaccard_distance
from sklearn.model_selection import train_test_split
from PIL import Image
from PIL import ImageOps
import fnmatch
import nibabel as nib
import shutil
from skimage.segmentation import mark_boundaries, find_boundaries
%matplotlib inline

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
def gather_images(data_path):
    images = []
    path = data_path + '\\'
    for f in os.listdir(data_path):
      if '_K' in f:
        continue
      elif '_C' in f:
        continue
      else:
        images.append(f)
        #segmentations.append(f.replace('.nii', '_K.nii'))

    #print(images[0], segmentations[0])
    images = np.array(images)
    #segmentations = np.array(segmentations)

    indices = np.array(range(len(images))) # we will use this in the next step.

    return images

In [3]:
def gather_set(data_path, phrase):
    set_of = []
    path = data_path + '\\'
    for f in os.listdir(data_path):
      if phrase in f:
        set_of.append(f)
      else:
        continue
    #set_of = np.array(set_of)

    indices = np.array(range(len(set_of))) # we will use this in the next step.

    return set_of

# Load in the model that was saved


# make sure to match the sizes and class numbers


This code assumes that the model is saved in the same location that this jupyter notebook is saved, if not make sure to put the full filepath in

In [4]:
model = custom_unet(input_shape=(512,512,1), num_classes=2)
#UNET_KU_EM_UB_train_MA_val
#UNET_MA_EM_UB_train_KU_val
#UNET_MA_KU_EM_train_UB_val
#UNET_MA_KU_UB_train_EM_val
#ALL_INSTITUTION_80-20
model.load_weights('UAB-kidneys-35.h5')
model_name = 'UAB-kidneys-35'

Define the path of the images that will be predicted on

this assumes there is one folder where all of the images are located with the masks defined by prefixes and subfixes

In [ ]:

#data_path = r'C:\Users\UAB\data\Emory\data'
#data_path = r'C:\Users\UAB\data\Mayo\data'
#data_path = r'C:\Users\UAB\data\UAB\data'
data_path = r'C:\Users\UAB\data\KU\data'
images = gather_images(data_path)

In [24]:
print(images[0])
print(len(images))
print(images[0:2])

KU_101934_0_96_L_0_M.npy
26082
['KU_101934_0_96_L_0_M.npy' 'KU_101934_0_96_L_10_M.npy']


# Specify if specific testing list - this will gather images left out for testing, you can also use random images if you want


In [25]:
image_list = []
#Emory
#phrase_list = ['283935R','290336L','290336R','295106L','295106R']
#Mayo
#phrase_list = ['380166R','383193L','383193R','385151L','385151R']
#UAB
#phrase_list = ['457036L','457036R']
#KU
phrase_list = ['113994L','139486R','157925R','183417R','186714L','186714R','187456L']

for i in range(len(phrase_list)):
    phrase1 = phrase_list[i][:-2]
    phrase2 = phrase_list[i][-1]
    print(phrase1, phrase2)
    for z in range(len(images)):
        name = images[z]
        if phrase1 in name:
            if phrase2 in name:
                image_list.append(name)
            else:
                continue
        else:
            continue

11399 L
13948 R
15792 R
18341 R
18671 L
18671 R
18745 L


In [26]:
print(len(images))
images = image_list
print(len(images))

26082
3117


In [27]:
d = {}
for i in images:
    if i not in d:
        d[i] = len(d)

labels_mapping = list(map(d.get, images))
#print(labels_mapping)

labels = {images[i]:labels_mapping[i] for i in range(len(images))}

In [28]:
print(len(images))

3117


## We need to gather a stack of 32 images for the prediction in the shape (32, 512, 512, 1)


In [ ]:
#divide the number of images avaliable into batches of 32, then we can feed it a batch at a time and 
# then we can append to the larger stack



label_list = []
predictions = []
pred_path = r"C:\Users\UAB\data\KU\data" #MAKE SURE TO DEFINE A SAVE FILE PATH
images = images

batches = len(images)//32
left_over = len(images)%32 
print(batches, left_over)


#REMEMBER TO COME BACK FOR LEFTOVERS
for i in range(batches):
    image_stack = np.empty((32,512,512,1))
    for x in range(32*i, (32*(i+1))):
        image = np.load(data_path +"\\"+images[x])
        y = x-(32*i)
        #print(x, y)
        image_stack[y, ...,0] = image[...]
        label_match = str(images[x][:-5] + 'K.npy')
        #print(label_match)
        label_list.append(label_match)
        
    img_prediction = model.predict(image_stack)
    img_prediction = img_prediction.astype('uint8')
    #print(img_prediction.shape)
    #print(img_prediction.shape)
    for z in range(len(img_prediction)):
        #print('saving images')
        img_numb = z + 32*i
        #print(z, img_numb)
        image_save = img_prediction[z,:,:]
        #image_save = find_boundaries(image_save, mode='thick').astype(np.uint8)
        label = label_list[img_numb][:-5]
        filename = str(label + model_name + '_K.npy')
        save_file = np.uint8(image_save[:,:,1])
        np.save(os.path.join(pred_path, filename),save_file)  


Get left over images predicted on

In [30]:
left_images = images[-left_over:]
print(len(left_images))
print(left_images[0])

13
KU_187456_3_120_L_89_M.npy


In [31]:
image_stack = np.empty((32,512,512,1))
label_left = []
for i in range(len(left_images)):
    image = np.load(data_path +"\\"+left_images[i])
    image_stack[i, ...,0] = image[...]
    label_match = str(left_images[i][:-5] + 'K.npy')
    label_left.append(label_match)
img_prediction = model.predict(image_stack)
img_prediction = img_prediction.astype('float')

for z in range(len(left_images)):
        #print('saving images')
        #print(z, img_numb)
        image_save = img_prediction[z,:,:]
        label = label_left[z][:-5]
        filename = str(label + model_name + '_K.npy')
        save_file = np.uint8(image_save[:,:,1])
        np.save(os.path.join(pred_path, filename), save_file)  

# Look at predicted images - optional

In [ ]:
data_path = r'C:\Users\UAB\data\UAB\AllNPY'
images = gather_images(data_path)

d = {}
for i in images:
    if i not in d:
        d[i] = len(d)

labels_mapping = list(map(d.get, images))
#print(labels_mapping)

labels = {images[i]:labels_mapping[i] for i in range(len(images))}

In [ ]:
model.load_weights('UAB-kidneys-35.h5')
model_name = 'UAB-kidneys-35'

In [ ]:
image_names = images[0:31]
image_stack = np.empty((32,512,512,1))
label_list = []
for i in range(len(image_names)):
    image = np.load(data_path +"\\"+images[i])
    image_stack[i, ...,0] = image[...]
    label_match = str(images[i][:-5] + 'K.npy')
    label_list.append(label_match)

print(image_stack.shape)
print(label_list)

In [ ]:
img_prediction = model.predict(image_stack)
img_prediction = img_prediction.astype('float')

In [ ]:
n = 5
img_prediction_30 = img_prediction[n,:,:,1]
plt.figure()
f, axarr = plt.subplots(nrows = 1, ncols = 3, figsize=(10,10))
f.tight_layout()

axarr[0].imshow(image_stack[n,:,:,0], cmap="gray")
axarr[0].title.set_text('Original Image')
axarr[1].imshow(og_seg, cmap='gray')
axarr[1].title.set_text('Truth Segmentation')
axarr[2].imshow(segmentation, cmap='gray')
axarr[2].title.set_text('Predicted Segmentation')

In [ ]:
def visualize_predicition(image_stack, n):
    fig, ax = plt.subplots()
    ax.imshow(image_stack[n,:,:,0], cmap=plt.cm.gray)
    og_seg = np.load(data_path +"\\"+label_list[n])
    segmentation = img_prediction[n,:,:,]
    segmentation = segmentation[:,:,1]

    contours = measure.find_contours(og_seg, 0.8)
    for j,contour in enumerate(contours):
        ax.plot(contour[:, 1], contour[:, 0],color='#FB3640', lw=2)
        contours = measure.find_contours(segmentation, 0.8)
        for contour in contours:
            ax.plot(contour[:, 1], contour[:, 0],color='#35A7FF', lw=2)

    ax.axis('image')
    ax.title.set_text('red is orignal - blue is prediction')

In [ ]:
visualize_predicition(image_stack, 15)